<a href="https://colab.research.google.com/github/IAI-UNSAM/UNSAM_ML/blob/main/20_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#The stars
import tensorflow as tf
from tensorflow import keras

#tensorboard
%load_ext tensorboard
import datetime, os 

In [ ]:
print(tf.__version__)
print(keras.__version__)
print(tf.config.list_physical_devices())

In [ ]:
# A small detail
tf.keras.backend.set_floatx('float32')

# Generative Adversarial Networks

## Data (Fashion_MNIST)

Just as `sklearn` allow loading standard datasets, so does `keras`.

In general, `keras` accepts datasets in the form of Numpy Arrays (like `sklearn`), but it also comes with a `Dataset` class that is optimized to load data (even if they are larger than the RAM memory of the computer).

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

As we know already, in large datasets CrossValidation can be very expensive, so it is recommended to leave apart a validation set from training.

We also normalize the pixels (ranging from 0 to 255) to be between 0 and 1.

In [ ]:
# Separo en entrenamiento y validacion, y normalizo los pixeles
# Los paso a 32 bits para que no haya problemas con TF.
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [ ]:
print(X_train_full.shape)

plt.imshow(X_train[0], cmap='binary', interpolation='None')

The target values are numeric, from 0 to 9. We can save the labels so it is easier for us to analyze how well or badly our model works:

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
# A useful function
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

## Define a GAN

The architecture is quite straight forward. We could even use the Sequential API from `keras` (**N.B.:** it is actually easier with the sequential API).

First define some hyperparameters

In [ ]:
keras.backend.clear_session()

In [ ]:
# Encoding size
codings_size = 30

# Generator
gen_unit_layers = [100, 150]

# Discriminator
dis_unit_layers = [150, 100]

# Training
discriminative_batches_per_generative_batch = 1
batch_size = 32

### Generator network

In [ ]:
# Define generator network as a decoder of a variational encoder
input_gen = keras.layers.Input(shape=(codings_size,), name='genin')

# Dense layers (iterate using hyperparameters from previous cell)
in_ = input_gen
for i, u in enumerate(gen_unit_layers):
    in_ = keras.layers.Dense(u, activation="selu", name='gen{}'.format(i + 1))(in_)

# We use sigmoid activation in the output layer to keep pixels in the range [0, 1]
# Reshape back to original size
flatgenout_ = keras.layers.Dense(28 * 28, activation="sigmoid", name='genout_flat')(in_)
out_gen = keras.layers.Reshape([28, 28], name='genout')(flatgenout_)

# Build full model
generator = keras.models.Model(input_gen, out_gen, name='Generative')

# generator = keras.models.Sequential([
#     keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
#     keras.layers.Dense(150, activation="selu"),
#     # We use sigmoid activation in the output layer to keep pixels in the range [0, 1]
#     # Reshape back to original size
#     keras.layers.Dense(28 * 28, activation="sigmoid"),
#     keras.layers.Reshape([28, 28])
# ])

In [ ]:
generator.summary()

### Disciminator network

In [ ]:
# The discriminator is a MLP for binary classification
input_dis = keras.layers.Input(shape=(28, 28), name='disin')
flatin_dis = keras.layers.Flatten(name='flat')(input_dis)

in_ = flatin_dis
for i, u in enumerate(dis_unit_layers):
    in_ = keras.layers.Dense(u, activation="selu", name='dis{}'.format(i + 1))(in_)

out_dis = keras.layers.Dense(1, activation='sigmoid', name='disout')(in_)

discriminator = keras.models.Model(input_dis, out_dis, name='Discriminative')

In [ ]:
discriminator.summary()

### Build GAN

In [ ]:
# Now use boh networks to from a GAN
# (thanks, keras API!)

# Build input layer
gan_input = keras.layers.Input(shape=(codings_size,), name='ganin')
# Generate new sample
H = generator(gan_input)
# Pass it to discriminator
gan_output = discriminator(H)

gan = keras.models.Model(gan_input, gan_output)
gan.summary()

### Training

Time to train the GAN. We have to propose a loss function for each of the two elements of the GAN.

The discriminating network is doing binary classification (no more, no less), so it is natural to use binary entropy.

On the other hand, the generating network is trained using both parts of the model, but without adjusting the weights of network D. Therefore, we have to do a little trick here, but is quite straightforward with `keras`.

We are going to use an optimizer a little more appropriate for this type of network: `rmsprop`

In [ ]:
# Compile discriminator
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")

# Define discriminator as non trainable (this will only be effective after compiling)
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In this way, the discriminator can be trained independently, but when we train the GAN network, its weights will remain fixed.

Now comes the tricky part, because we can't train the network with a simple call to the `fit` method. 

First we generate a data set, of size `batch_size`.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

And now we write a adversarial training routine (Rocky theme plays).

In [ ]:
# Codigo de Géron (https://github.com/ageron/handson-ml2)

def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50,
              dbpgb=discriminative_batches_per_generative_batch, tensorboard=None):
    generator, discriminator = gan.layers[1:]
    
    # Transform train_on_batch return value
    # to dict expected by on_batch_end callback
    def named_logs(model, logs):
      result = {}
      for l in zip(model.metrics_names, logs):
        result[l[0]] = l[1]
      return result
    
    # ADd model to Tensorboard
    #if tensorboard is not None:
    #    tensorboard.set_model(gan)
        
    for epoch in range(n_epochs):
        if tensorboard is not None:
            tensorboard.set_model(gan)
    
        print("Epoch {}/{}".format(epoch + 1, n_epochs))              # not shown in the book
        for i, X_batch in enumerate(dataset):
            #
            # Phase 1 - train the discriminator
            #
            
            # Create noise (samples) drawn from an isotropic multinormal
            # with same dimension as coding_size
            # 
            # Genera ruido (muestras) salido de una multinormal isotrópica con 
            # tamaño igual al coding_size
            codes = tf.random.normal(shape=[batch_size, codings_size])
            
            # Run generator to produce fake images
            # Corre el generador para producir imágenes ficticias
            generated_images = generator(codes)
            
            # Build dataset combining these images with real ones
            # Genera un dataset combinando estas imágenes con imágenes reales
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            
            # Make labels to identify the real ones from the fake ones
            # Genera las etiquetas que identifican como verdadero o falso
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            
            # Esta línea es para evitar un warning, el discriminador ya es
            # entrenable
            # discriminator.trainable = True
            
            # Train the discriminator
            # Entrena al discriminador
            discriminator.train_on_batch(X_fake_and_real, y1)

            # Every generative_batches_per_discriminative_batch, train also G
            if (i+1) % dbpgb == 0:
                #
                # Phase 2 - train the generator
                #
                # More noise (they are actually samples in the latent space!)
                # Más ruido (en realidad muestras en el espacio latente!)
                codes = tf.random.normal(shape=[batch_size, codings_size])

                # Labels (this time, true for all)
                y2 = tf.constant([[1.]] * batch_size)

                # Otra vez, para evitar un warning
                # discriminator.trainable = False

                # Train full model (generator + discriminator) with these labels

                # Entrena todo el modelo: generador + discriminador, con esas 
                # etiquetas
                logs = gan.train_on_batch(codes, y2)
        
                #
                # Tensorboard
                if tensorboard is not None:
                    tensorboard.on_epoch_end(epoch, named_logs(gan, [logs,]))

        if tensorboard is not None:
            tensorboard.on_train_end(None)

        plot_multiple_images(generated_images, 8)                     # not shown
        plt.show()                                                    # not shown

We are going to train it and see the images generated by the G network.

In [ ]:
#tensorboard initialization
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
%tensorboard --logdir logs

In [ ]:
# tensorboard_callback.set_model(generator)

In [ ]:
train_gan(gan, dataset, batch_size, codings_size, n_epochs=2)

In [ ]:
# Now we train for longer, and use TensorBoard to follow the evolution.
train_gan(gan, dataset, batch_size, codings_size, n_epochs=50, tensorboard=tensorboard_callback)

In [ ]:
gan.save('models/gan_model1.h5')

## Deep Convolutional GAN (DCGAN)

Of course, we can do a similar (better) job using CNNs.

In [ ]:
# Codigo de Géron (https://github.com/ageron/handson-ml2)
tf.random.set_seed(42)
np.random.seed(42)

codings_size = 100

generator = keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 128, input_shape=[codings_size]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="SAME",
                                 activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="SAME",
                                 activation="tanh"),
])
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2),
                        input_shape=[28, 28, 1]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")
])
dcgan = keras.models.Sequential([generator, discriminator])

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
dcgan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
X_train_dcgan = X_train.reshape(-1, 28, 28, 1) * 2. - 1. # reshape and rescale

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train_dcgan)
dataset = dataset.shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [ ]:
train_gan(dcgan, dataset, batch_size, codings_size)

## Let's discuss!

1. Come up with at least a couple of ways you could use GANs for you work or for the company.
2. Compare and contrast this generative method with another generative models.

# Mixture Density Networks (MDN)
<a id='mixtures'></a>

## A simple regression problem...

Cuando estudiamos regresión lineal, vimos que el error cuadrático aparecía naturalmente al suponer que la distribución condicional de los labels era normal:

$$
p(t|x, \mathbf{w}, \beta) = \mathcal{N}(t|y(x,\mathbf{w}), \beta^{-1})\;\;.
$$

En ese caso, teníamos que el logaritmo de la versosimilitud de una serie de mediciones, $\mathbf{X} = \{\mathbf{x}_1, \ldots, \mathbf{x}_N\}$, con labels / outputs, $\mathbf{t} = \{t_1, \ldots, t_N\}$ era:

$$
\begin{array}{rcl}
\ln p(\mathbf{t}|\mathbf{w}, \beta) &=& \sum_{n=1}^N \ln\mathcal{N}(t_n|y(\mathbf{x},\mathbf{w}), \beta^{-1})\\
&=& \frac{N}{2}\ln\beta - \frac{N}{2}\ln(2\pi) - \beta E_D(\mathbf{w})\;\;,
\end{array}
$$
donde 
$$
E_D(\mathbf{w}) = \frac{1}{2}\sum_{n=1}^N\left\{t_n - \mathbf{w}^T\phi(\mathbf{x}_n)\right\}^2 \;\;.
$$

En el marco de los modelos de redes neuronales, las funciones de base $\phi$ se parametrizan para darle mayor flexibilidad al modelo, como estuvimos viendo estas semanas. 

Muy bien, veamos entonces qué pueden hacer las redes neuronales frente a un problema sencillo de regresión, con un output y un input. Seguimos de cerca a Bishop.

In [ ]:
# Fix seed
np.random.seed(20200616)

# Create dataset
x = np.random.rand(600).reshape(-1, 1)
t = 1.0 * x + 0.3 * np.sin(2 * np.pi * x)

noise = (np.random.rand(x.shape[0]) * 0.2 - 0.1).reshape(-1, 1)

t += noise
# Separemos en train, test, y validation
X_train, X_validation, X_test = x[:500], x[500: 550], x[550:]
t_train, t_validation, t_test = t[:500], t[500: 550], t[550:]

# Veamos los datos
plt.plot(X_train, t_train, '.')

In [ ]:
#
from tensorflow import keras

# Build simple DNN
modelo = keras.Sequential()

# Input layer
modelo.add(keras.layers.InputLayer(input_shape=X_train.shape[1:]))

# Agreguemos dos capas ocultas de 10 y 8 neuronas cada una
modelo.add(keras.layers.Dense(10, activation='sigmoid', name='Oculta1'))
modelo.add(keras.layers.Dense(8, activation='sigmoid', name='Oculta2'))

# Agregemos una capa de output que sirva para este problema
modelo.add(keras.layers.Dense(1, activation=None))

# keras.utils.plot_model(modelo, show_shapes=True)

In [ ]:
modelo.summary()

In [ ]:
# Compilemos el modelo. 
# En este punto se elije la función de périda y el optimizador.
# ¿Qué función de pérdida deberíamos usar?
modelo.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.1))

In [ ]:
# Es hora de entrenar el modelo. Corramos 500 épocas, y vayamos mirando también
# la validación.

# Usemos EarlyStopping para no pasar tiempo de más entrenando
from tensorflow.keras.callbacks import EarlyStopping

# Set up early stopping
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=50, verbose=0, 
                      mode='auto')

history = modelo.fit(X_train, t_train, epochs=200, 
                     validation_data=(X_validation, t_validation),)
                     #callbacks=[early,])

In [ ]:
# Veamos el resultado del ajuste
import pandas as pd

df = pd.DataFrame(history.history)
df.plot()
ax = plt.gca()
# ax.set_ylim(0.01, 0.025)

In [ ]:
# Veamos los datos
plt.plot(X_train, t_train, '.')

# Graficamos la predicción del modelo 
xx = np.linspace(0, 1, 1000)
plt.plot(xx, modelo.predict(xx), '-r')

In [ ]:
modelo.layers[-1].get_weights()

In [ ]:
# modelo.save('univariate_regression.h5')

Vemos que el modelo funciona bien (hay que prestar atención al learning rate, nomás).

## ... that suddenly becomes interesting.

Ahora veamos qué pasa si invertimos el rol de las variables, de manera que los datos que queremos ajustar son diferentes.

In [ ]:
# Veamos los datos
plt.plot(t_train, X_train, '.')

In [ ]:
# Escribamos un modelo similar al anterior (¿podemos usar el mismo modelo?)
# Build simple DNN
modelo2 = keras.Sequential()

# Input layer
modelo2.add(keras.layers.InputLayer(input_shape=t_train.shape[1:]))

# Agreguemos dos capas ocultas de 10 y 8 neuronas cada una
modelo2.add(keras.layers.Dense(10, activation='sigmoid', name='Oculta1_2'))
modelo2.add(keras.layers.Dense(8, activation='sigmoid', name='Oculta2_2'))

# Agregemos una capa de output que sirva para este problema
modelo2.add(keras.layers.Dense(1, activation=None))

In [ ]:
# Compilemos, ajustemos, miremos el resultado y grafiquemos la predicción.
modelo2.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.1))

# keras.utils.plot_model(modelo2, show_shapes=True)

In [ ]:
# OJO! OJO! En el validation_data, tiene que ir una tupla (no una lista!); 
# es decir, paréntesis y no corchetes
history2 = modelo2.fit(t_train, X_train, epochs=500, 
                       validation_data=(t_validation, X_validation))

# También se puede pasar una fracción del conjunto de entrenamiento para usar
# como validación.
# modelo2.fit(t_train, X_train, epochs=50, validation_split=0.1)

In [ ]:
plt.plot(t_train, X_train, '.')

plt.plot(xx, modelo2.predict(xx), 'r-')

Por más que intentemos, la red no va a poder ajustar los datos. 

El uso del error cuadrático implica, como vimos arriba que la distribución condicionada de las variables target $p(t | x)$ es una normal. Sin embargo, para valores como $x=0.6$ la distribución es bien multimodal, y por lo tanto no puede ser representada por una normal.

In [ ]:
# Veamos un entorno de x = 0.6
epsilon = 0.05
cond = np.abs(t_train - 0.6) < epsilon

fig = plt.figure(figsize=(16, 6))
ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax.plot(t_train, X_train, '.k')
ax.axvline(0.6, color='r', ls='-')
for sign in (-1, 1):
    ax.axvline(0.9 + sign * epsilon, color='r', ls=':')
ax2.hist(X_train[cond], 25)

ax.set_xlabel('nuevo X')
ax.set_ylabel('nuevo T')
ax2.set_xlabel('nuevo X')


We clearly cannot fit this using a normal distribution.

## Model presentation

Este ejemplo muestra una de las limitaciones de un modelo normal: solo puede captar distribuciones con un único modo. Una forma de sobreponerse a este problema es usar un modelo que consista en una superposición de $K$ distribuciones normales.

$$
p(\mathbf{x}) = \sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\;\;,
$$
donde cada densidad $\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$ se conoce como una *componente* de la mixtura, y tiene su propio valor medio y matriz de covarianza. Los $\pi_k$ son los *coeficientes* de mezcla.

__Los coeficientes como probabilidades__

Integrando a ambos lados de la igualdad sobre todos los valore de $\mathbf{x}$ y viendo que tanto $p(\mathbf{x})$ como las componentes normales son funciones de distribución de probabilidad, llegamos a que

$$
\sum_{k=1}^K \pi_k = 1\;\;.
$$

Además, como $p(\mathbf{x})$ tienen que ser positiva, encontramos que una forma de garantizar eso de manera general es definir que los coeficientes sean positivos. Por lo tanto, $0 \leq \pi_k \leq 1$, y los coeficientes satisfacen los requerimientos para ser probabilidades.

Podemos, entonces, asignar una función de distribución de masa, $p(k)$, que describe la probabilidad (prior) de que se elija la compoenente $k$, y $p(k) = \pi_k$. Entonces, distribución sobre $\mathbf{x}$ queda:

$$
p(\mathbf{x}) = \sum_{k=1}^K p(\mathbf{x}, k) = \sum_{k=1}^K p(k) p(\mathbf{x} | k)\;\;,
$$

donde la probabilidad condicionada $p(\mathbf{x} | k) = \mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$.

In [ ]:
import scipy.stats as st

# Supongamos una distribución unidimensional de dos modos, con los siguientes parámetros
mu1 = [0.0,]
sigma1 = [2.0,]
mu2 = [5.0,]
sigma2 = [1.5,]

# Generemos las dos distris
n1 = st.norm(mu1, sigma1)
n2 = st.norm(mu2, sigma2)

# Ahora pensemos que tenemos una distribución prior p(k) con las siguientes elementos
p1 = 0.65
p2 = 0.35

# Veamos como podemos generar, fácilmente, muestras de la mixtura:
# p1 * N1 + p2 * N2
# Primero muestreo p(k)
N = 5000
k = np.where(np.random.rand(N) < p1, 1, 2)

# Y ahora lo combino con las mixturas

# Modo 1 (estricto)
# x1 = n1.rvs(sum(k==1), 1)
# x2 = n2.rvs(sum(k==2), 1)

# x = np.hstack([x1, x2])

# Modo 2 (generando de más)
x1 = n1.rvs(N, 1)
x2 = n2.rvs(N, 1)

x = np.where(k==1, x1, x2).reshape((-1, 1))

h = plt.hist(x, 50, histtype='step', density=True)

# Combinemos esto con la pdf del modelo
xx = np.linspace(x.min(), x.max(), 400)
plt.plot(xx, p1 * n1.pdf(xx) + p2 * n2.pdf(xx), 'r-')

__Responsabilidades__

Un rol crucial lo van a jugar las posteriores de la probabilidad de cada modo $p(k \:|\: \mathbf{x})$, que se conocen como *responsabilidades* (¡cuánta seriedad!)

$$
\gamma_k(\mathbf{x}) \equiv p(k \:|\: \mathbf{x})\;\;.
$$

Como es de esperar, podemos calcular la punta que tienen las responsabilidades a partir del teorema de Bayes:

$$
\begin{eqnarray}
\gamma_k(\mathbf{x}) &=& \frac{p(k) p(\mathbf{x}\:|\:k)}{\sum_i p(i) p(\mathbf{x}\:|\:i)}\\
   &=&\frac{\pi_k{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}}{\sum_i{\pi_i{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_i, \boldsymbol{\Sigma}_i)}}}\;\;,
\end{eqnarray}
$$
donde usamos la definición de arriba
$$
p(\mathbf{x}) = \sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\;\;.
$$

__Verosimilitud__ (o Donde arrancan los líos)

Naturalmente, necesitaremos poder escribir la verosimilitud para un modelo de este tipo. Como vimos arriba, el problema con las exponenciales de la normal, es que muchas veces generan problemas numéricos.

Vamos entonces a escribir el logaritmo de la verosimilitud, como ya hicimos un montón de veces.

$$
\ln p(\mathbf{X} \:|\: \boldsymbol{\mu}, \boldsymbol{\Sigma}, \boldsymbol{\pi}) = \sum_{n=1}^N \ln \left\{\sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x}_n \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\right\}\;\;,
$$
donde estamos usando esta notación: $\boldsymbol{\mu} \equiv \left\{\boldsymbol{\mu}_1, \ldots, \boldsymbol{\mu}_K\right\}$, y lo mismo para $\boldsymbol{\Sigma}$ y para $\boldsymbol{\pi}$. Y como siempre $\mathbf{X} = \left\{\mathbf{x}_1, \ldots, \mathbf{x}_N\right\}$.

Vemos que el logaritmo actúa sobre la sumatoria y no sobre cada normal de manera individual. Esto hace que sea complicado obtener una forma cerrada para los parámetros que maximizan la verosimilitud. Tenemos que usar un procedimiento iterativo, conocido como el algoritmo de maximización de la expectación, o *expectation maximisation* (EM).


__Un paso más__

En el contexto de las redes neuronales, vamos a dar un paso más y permitir a los parámetros de cada componente que cambien con la variable del input, $\mathbf{x}$.

Así, tanto los coeficientes de mezcla, como los vectores $\boldsymbol{\mu}$, como la varianza de cada normal (que consideraremos isotrópica para cada $\mathbf{x}$, se convierten en funciones del input.

El modelo pasa de ser una normal con covarianza general:

$$
p(t|\mathbf{x}) = \mathcal{N}(t|y(\mathbf{x},\mathbf{w}), \beta^{-1})\;\;.
$$

a ser una mixtura de normales isotrópicas (si bien la extensión a covarianzas generales es posible):

$$
p(t | \mathbf{x}) = \sum_{k=1}^K \pi_k(\mathbf{x}){\mathcal{N}(t \:|\: \boldsymbol{\mu}_k(\mathbf{x}), \sigma^2_k(\mathbf{x}))}\;\;.
$$

Esta formulación es un ejemplo de un modelo heterocedastico (wow!); es decir, que la varianza cambia con la variable $x$. 

## Implementation

Todo esto es muy lindo, pero ¿cómo se implementa en un modelo de redes neuronales?

La idea es generar una red cuyos outputs sean las parámetros del modelo. Para eso, tenemos que usar la API funcional de `Keras`

**Nota**: Nos inspiramos en el código de https://github.com/oborchers/Medium_Repo

[](https://drive.google.com/uc?export=view&id=1-a9oAqcGYPgBoSkg9yTXfzxsu5MBjdA5)


<img src='./images/mixture_density_model.png'>

In [ ]:
from tensorflow.keras.layers import Input, Dense, Concatenate

neurons = 500     # Neurons of the DNN hidden layers
components = 3    # Number of components in the mixture
no_parameters = 3 # Paramters of the mixture (pi, mu, sigma)

input_ = Input(shape=(t_train.shape[1],))

# Escribir una o dos capas ocultas
h1 = Dense(neurons, activation='relu')(input_)
h2 = Dense(neurons, activation='relu')(h1)

# Ahora escribir una capa para cada una de las variables del problema
# Coeficientes de mezcla (que tienen que sumar 1)
# Valores medios
# Anchos de la distribución (que tiene que ser positivos)
mixcoeff = Dense(components, activation='softmax', name='pi')(h2) 
means = Dense(components, activation=None, name='mu')(h2)
sigmas = Dense(components, activation='exponential', name='sigma')(h2)

# Concatenemos las tres capas para dar la capa de salida
pvector = Concatenate(name="output")([mixcoeff, means, sigmas])

# Y finalmente, generemos el modelo de Keras con todo esto
modelMDN = keras.Model(inputs=[input_], outputs=[pvector])

In [ ]:
# Si quisieramos, podemos hacer un subclase de los modelos de Keras
class MDN(keras.Model):

    def __init__(self, neurons=100, components = 2):
        super(MDN, self).__init__(name="MDN")
        self.neurons = neurons
        self.components = components
        
        self.h1 = Dense(neurons, activation="relu", name="h1")
        self.h2 = Dense(neurons, activation="relu", name="h2")
        
        self.mixcoeff = Dense(components, activation="softmax", name="mix")
        self.means = Dense(components, name="means")
        self.sigmas = Dense(components, activation="exponential", name="sigmas")
        self.pvec = Concatenate(name="pvec")
        
    def call(self, inputs):
        x = self.h1(inputs)
        x = self.h2(x)
        
        mix_v = self.mixcoeff(x)
        mean_v = self.means(x)
        sigma_v = self.sigmas(x)
        
        return self.pvec([alpha_v, mu_v, sigma_v])

¡Bien! Ya tenemos el modelo. Pero eso es solo una parte del problema.

¿Qué más falta?

---

In [ ]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd

def slice_parameter_vectors(parameter_vector):
    """ Returns an unpacked list of paramter vectors.
    """
    return [parameter_vector[:,i*components:(i+1)*components] for 
            i in range(no_parameters)]

def gnll_loss(t, parameter_vector):
    """ Computes the mean negative log-likelihood loss of y given the mixture parameters.
    """
    pi, mu, sigma = slice_parameter_vectors(parameter_vector)  # Unpack parameter vectors
    
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=pi),
        components_distribution=tfd.Normal(
            loc=mu,       
            scale=sigma))
    
    log_likelihood = gm.log_prob(tf.transpose(t))                 # Evaluate log-probability of y
    
    return -tf.reduce_mean(log_likelihood, axis=-1)

### Compilation and Training

In [ ]:
modelMDN.compile(loss=gnll_loss, optimizer=keras.optimizers.Adam(lr=0.002))
modelMDN.summary()

In [ ]:
keras.utils.plot_model(modelMDN, show_shapes=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Set up early stopping
mon = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=50, verbose=0, 
                    mode='auto')

In [ ]:
# Ajustemos el modelo. Volvemos a usar early stopping

historyNEW = modelMDN.fit(t_train, X_train, epochs=1000, 
                          validation_data=(t_validation, X_validation), 
                          verbose=False, batch_size=16, callbacks=[mon,])

In [ ]:
import pandas as pd
pd.DataFrame(historyNEW.history).plot()

In [ ]:
# Veamos los parámetros del resultado

fig = plt.figure(figsize=(16, 6))
ax = fig.add_subplot(141)
ax2 = fig.add_subplot(142)
ax3 = fig.add_subplot(143)
ax4 = fig.add_subplot(144)

# En el primer subplot ponemos los datos.
ax.plot(t_train, X_train, '.k')
ax.set_xlabel('nuevo X')
ax.set_ylabel('nuevo T')

# Ahora calculamos la predicción.
# En este caso, no tenemos una predicción directa de los datos,
# sino los parámetros de la mixtura
xx = np.arange(0, 1.001, 0.001)
pars = modelMDN.predict(xx)
pi, mu, sigma = slice_parameter_vectors(pars)

# Podemos calcular el valor medio de la función y ponerla con los datos
# El valor medio es la suma de pi * mu sobre las componentes del modelo
tmean = np.sum(pi * mu, axis=1)
ax.plot(xx, tmean, ',')
# ¿Sorprende el resultado?

ax2.plot(xx, pi)
ax2.set_title('Mixing coefficient')
ax3.plot(xx, mu)
ax3.set_title('Valor medio')
ax4.plot(xx, sigma)
ax4.set_title('Varianza')

for a in [ax2, ax3, ax4]:
  a.set_xlabel('nuevo X')

In [ ]:
# Ahora la posta. 
# Calculemos la pdf de la distribución condicional, para una serie de valores
# de X.
import scipy.stats as st

# Primero construyo una grilla de valores
# uso xx para ambos en este caso porque los valores están todos en [0, 1].
x, y = np.meshgrid(xx, xx)

# Calcula distribución para cada componente
a = np.array([pi[:, i] * st.norm.pdf(y, loc=mu[:, i], scale=sigma[:, i]) for 
              i in range(pi.shape[1])])

# Sumo sobre todas las componentes
pdf = np.sum(a, axis=0)

In [ ]:
pdf.shape

In [ ]:
# La hora de la verdad.
# Graficamos el contorno de la distribución de mixtura para distintos 
# valores de x. Es decir, p(t | x) con x que va variando.

xont = plt.contourf(x, y, pdf, 30, cmap='jet')
xont = plt.contour(x, y, pdf, 15, colors=['w',], alpha=0.7)

# Y agregamos los datos
plt.plot(t_train, X_train, '.w', mfc='None', alpha=0.4)

In [ ]:
# También podemos mostrar el contorno de cada componente gaussiana.
# xont = plt.contour(x, y, pdf, 15, colors=['m',], alpha=0.7, lw=5)
for k in range(a.shape[0]):
  plt.contour(x, y, a[k], 15, colors=['C{}'.format(k)])

# plt.colorbar(xont)
# plt.plot(t_train, X_train, '.w', mfc='None', alpha=0.4)

In [ ]:
# Quedó bueno. Salvémoslo.
modelMDN.save('MDN_ok.h')

In [ ]:
import matplotlib.pyplot as plt
# Usemos el resultado de arriba para plotear la función condicionada para algunos
# valores de x. Es decir, hacemos cortes verticales para valores de x fijos.
xs = [0.1, 0.4, 0.7]

fig = plt.figure(figsize=(9,6))

for i, xi in enumerate(xs):
  # Nos quedamos con el índice correspondiente al valor de x elegido
  j = np.argmin(np.abs(xx - xi))
  ax = fig.add_subplot(1, len(xs), i+1)
  
  # Plot de la distribución condicional p(t | x=xi)
  ax.plot(xx, pdf[:, j].ravel(), '-k', lw=2)
  
  # Aporte de cada componente
  for k in range(a.shape[0]):
    ax.plot(xx, a[k, :, j], alpha=0.6, lw=5)
  
  ax.set_title('p(t | X={}, wML)'.format(xi))